<a href="https://colab.research.google.com/github/NJiHyeon/Pytorch_for-deep-learning/blob/main/CH3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[필요한 모듈 설치 및 설명]
###### 1. torchvision.transforms는 데이터를 불러오면서 그 다음에 전처리를 바로 할 수 있게 해주는 라이브러리
###### 2. DataLoader는 배치 사이즈 형태로 만들어서 실제로 학습을 할 때 이용할 수 있는 형태를 만들어주는 라이브러리
###### 3. dataset은 튜닝을 할 때 사용

In [2]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np

🔎파이토치 제공 데이터 불러오기

1.transform 정의

In [3]:
transf = tr.Compose([tr.Resize(8), tr.ToTensor()])



###### - Compose : 전처리를 할 때 순서대로 작업을 수행하게 되는 것(위의 같은 경우, 8 by 8fh resize가 되고, 텐서 데이터로 바꿔주는 것)
###### -  처음에 들어오는 이미지는 Transforms on PIL Image라고 해서 특정 타입을 말하는 것
###### -  종류 : Pad, Grayscale, RandomCrop, Normalize ..
###### -  Transforms on torch.*Tensor - tensor image




2.data 불러오기

In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transf)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
trainset[0][0].size()

torch.Size([3, 8, 8])



###### - trainset의 size 확인(첫번째꺼만 불러오고 튜플형태로 되어있다.(이미지,레이블))
###### - 결과 : 채널 3개에 8 by 8 이미지



In [6]:
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)


###### - DataLoader를 이용해서 각각을 정의
###### - num_workers는 data를 load할 때 subprocess를 몇개 쓰느냐를 정의
###### - 이 단계까지 하면 배치 형태로 모두 분리 해놓은 상태가 된다.






In [8]:
len(trainloader)

1000



###### - CIFAR 데이터의 개수가 50000개인데 batch_size가 50이므로 trainloader의 길이가 1000가 된다.



In [9]:
dataiter = iter(trainloader)
images, labels = dataiter.next()



###### - 실제 값을 보고 싶을 때 



In [11]:
images.size()

torch.Size([50, 3, 8, 8])



###### - 파이토치는 신경망에 들어갈 때 배치사이즈, 채널 수, 이미지 사이즈의 순서로 들어간다. 


✅ transforms, torchvision.sets, DataLoader 3줄이면 끝 



---



🔎같은 클래스 별 폴더 이미지 데이터 이용


In [17]:
# ./class/tiger ./class/lion
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)
trainloader = DataLoader(trainset, batch_size=10, shuffle=False, num_workers=2)
print(len(trainloader))

FileNotFoundError: ignored

###### - torchvision.datasets.ImageFolder를 이용하면 class안에 있는 이미지들을 알아서 search해주고 각각의 다른 폴더에 대해서 labeling을 자동으로 다르게 매겨준다.
###### - 전처리 또한 이용할 수 있다.(transform=transf)
###### - trainset= 한줄로 데이터 전체를 모두 불러오면서 레이블이 자동으로 매겨지면서 전처리까지 가능


In [ ]:
trainset[0][0].size()



---



🔎개인 데이터 사용

In [18]:
#import preprocessing

train_images = np.random.randint(256, size=(20,32,32,3))
train_labels = np.random.randint(2, size=(20,1))

#preprocessing ...
#train_images, train_labels = preprocessing(train_images, train_labels)

print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [19]:
class TensorData(Dataset) :
  def __init__(self, x_data, y_data) :
    self.x_data = torch.FloatTensor(x_data) #tensor 변환(구체적인 tensor)
    self.x_data = self.x_data.permute(0,3,1,2)  #이미지 개수, 채널 수, 이미지 너비, 높이 순서 바꾸기
    self.y_data = torch.FloatTensor(y_data)
    self.len = self.y_data.shape[0]

  def __getitem__(self, index) :
      return self.x_data[index], self.y_data[index] #__getitem__으로해서 x,y를 튜플형태로

  def __len__(self) :
    return self.len #데이터 개수 산출

###### - dataset의 class에 대해 상속을 받을 class를 만들기
###### - 상속받을 클래스의 이름은 원하는대로 지정

In [21]:
train_data = TensorData(train_images, train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
train_data[0][0].size()

torch.Size([3, 32, 32])

###### - class 인스턴스 생성(train_images, train_labels)
###### - 데이터가 만들어지면 배치 형태로 만들어야 하니까 DataLoader에 다시 넣어주는 작업 후 사이즈 확인

In [22]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [23]:
images.size()

torch.Size([10, 3, 32, 32])

🔎개인적으로 torchvision.datasets.ImageFolder를 쓰지 않는 이유

###### - 3줄안에 못 만드는 경우가 있다.
###### - 예를 들어 다른 작업에도 쓰는 데이터의 경우와 폴더가 아닌 SQL 같은 곳에서 넘어오는 경우